<a href="https://colab.research.google.com/github/Filarh/Cringe-scraper/blob/main/run_in_colab_traducido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!git clone https://github.com/NianBroken/Qwen_Fine-tuning.git
%cd Qwen_Fine-tuning
!pip install -r requirements.txt

In [ ]:
import os
from pathlib import Path

# Ajusta aquí el nombre del subdirectorio donde quieres guardar tu adaptador:
output_subdir = "output/qwen-ft"

# Construye la ruta absoluta y créala si no existe
adapter_path = Path(output_subdir)
adapter_path.mkdir(parents=True, exist_ok=True)

# Convierte a cadena para usar en el comando
adapter_path = str(adapter_path)

print(f"✅ Ruta de salida creada en: {adapter_path}")

In [ ]:
# Celda: Combinar datasets al formato de "messages"
import json

def convert_formateado(line):
    rec = json.loads(line)
    return {
        "messages": [
            {"role": "system",    "content": "Eres un asistente útil."},
            {"role": "user",      "content": rec["title"]},
            {"role": "assistant", "content": rec["description"]}
        ]
    }

def convert_rare(line):
    rec = json.loads(line)
    # Construir el contenido de usuario a partir de instruction e input
    user_content = rec["instruction"]
    if rec.get("input"):
        # Si hay input, lo concatenamos en nueva línea
        user_content += "\n" + rec["input"]
    return {
        "messages": [
            {"role": "system",    "content": "Eres un asistente útil."},
            {"role": "user",      "content": user_content},
            {"role": "assistant", "content": rec["output"]}
        ]
    }

# Archivos de entrada y salida
input_files = [
    ("/content/Qwen_Fine-tuning/datos-instruct-formateado.jsonl", convert_formateado),
    ("/content/Qwen_Fine-tuning/rare_instruction_finetune.jsonl", convert_rare)
]
output_path = "combined_dataset.jsonl"

with open(output_path, "w", encoding="utf-8") as fout:
    for fname, converter in input_files:
        with open(fname, "r", encoding="utf-8") as fin:
            for line in fin:
                line = line.strip()
                if not line:
                    continue
                new_rec = converter(line)
                fout.write(json.dumps(new_rec, ensure_ascii=False) + "\n")

print(f"✅ Dataset combinado guardado en `{output_path}`")


# 🔧 Ajuste fino de Qwen con EchoHeart en Google Colab

Este notebook configura automáticamente el entorno, inicia el entrenamiento de ajuste fino (QLoRA), realiza pruebas, fusiona el adaptador LoRA con el modelo base y exporta el modelo final en formato GGUF.

### 📌 Configuración
Para comenzar, especifica el modelo base y el conjunto de datos modificando las variables en el primer bloque de código más abajo.

---

### 🪜 Pasos del proceso

1. ✅ **Configurar variables, clonar o actualizar el repositorio de GitHub y definir rutas.**  
2. 📦 **Instalar las dependencias necesarias.**  
3. 🧠 **Ejecutar el script de entrenamiento con QLoRA.**  
4. 🧪 **(Opcional)** Ejecutar un script de prueba para interactuar con el modelo afinado (adaptador).  
5. 🔀 **Fusionar el adaptador LoRA con el modelo base.**  
6. 📤 **(Opcional)** Exportar el modelo fusionado al formato GGUF para despliegue eficiente.


In [ ]:
#@title 🔧 Configuración principal y parámetros de entrenamiento
#@markdown ### ⚙️ Modelo y archivo de datos
base_model_name: str = "Qwen/Qwen2.5-1.5B-Instruct"  #@param {type:"string"}
#@markdown *Nombre o ruta del modelo base en Hugging Face (por ejemplo: `"Qwen/Qwen2.5-7B-Instruct"`)*

dataset_file: str = "/content/Qwen_Fine-tuning/combined_dataset.jsonl"  #@param {type:"string"}
#@markdown *Ruta relativa al archivo JSONL del dataset a usar para el fine-tuning*

#@markdown ---
#@markdown ### 🧪 Hiperparámetros de entrenamiento
num_train_epochs: int = 2  #@param {type:"integer"}
#@markdown *Número de épocas (vueltas completas al dataset)*

learning_rate: float = 0.0002  #@param {type:"number"}
#@markdown *Tasa de aprendizaje inicial*

weight_decay: float = 0.01  #@param {type:"number"}
#@markdown *Factor de decaimiento del peso (weight decay) para evitar overfitting*

max_grad_norm: float = 1.0  #@param {type:"number"}
#@markdown *Límite al valor de los gradientes (grad clipping)*

seed: int = 42  #@param {type:"integer"}
#@markdown *Semilla aleatoria para reproducibilidad*

#@markdown ---
#@markdown ### 🧩 Parámetros de LoRA
lora_r: int = 32  #@param {type:"integer"}
#@markdown *Rango (rank) para LoRA*

lora_alpha: int = 64  #@param {type:"integer"}
#@markdown *Alpha (factor de escalado) para LoRA*

lora_dropout: float = 0.05  #@param {type:"number"}
#@markdown *Dropout aplicado dentro de LoRA*

#@markdown ---
#@markdown ### 💾 Guardado y registros
save_steps: int = 25  #@param {type:"integer"}
#@markdown *Guardar un checkpoint cada N pasos*

logging_steps: int = 5  #@param {type:"integer"}
#@markdown *Registrar información cada N pasos*

#@markdown ---
#@markdown ### 🗂️ Ruta personalizada de salida (opcional)
custom_output_dir: str = ""  #@param {type:"string"}
#@markdown *Ruta relativa personalizada para guardar el adaptador entrenado (dejar vacío para generar automáticamente)*

#@markdown ---
#@markdown ### 📁 Directorio de trabajo base
target_workspace_dir: str = "/content"  #@param {type:"string"}
#@markdown *Directorio base donde se clonará el repositorio (en Colab se ajusta automáticamente a `/content`)*


In [ ]:
# 3. Ejecutar script de entrenamiento (QLoRA)
print("Starting QLoRA training...")

train_command = (
    f"python train.py "
    f"--base_model_name \"{base_model_name}\" "
    f"--dataset_file \"{dataset_file}\" "
    f"--output_dir \"{adapter_path}\" "
    f"--num_train_epochs {num_train_epochs} "
    f"--learning_rate {learning_rate} "
    f"--weight_decay {weight_decay} "
    f"--max_grad_norm {max_grad_norm} "
    f"--lora_r {lora_r} "
    f"--lora_alpha {lora_alpha} "
    f"--lora_dropout {lora_dropout} "
    f"--save_steps {save_steps} "
    f"--logging_steps {logging_steps} "
    f"--seed {seed}"
)

print("\n--- Running Training Command ---")
print(train_command)
print("------------------------------\n")

# Ejecutar comando
!{train_command}

## ¡Entrenamiento completado!

El modelo ajustado se guarda en el directorio `output/qwen-ft` del sistema de archivos del entorno Colab.


In [ ]:
# 4. 运行测试脚本
print("Starting non-interactive testing session...")
!python test_model.py --base_model_name "{base_model_name}" --adapter_path "{adapter_path}"

## 5. (Opcional) Exportar al formato GGUF

El archivo convertido se guardará en la ruta: `{gguf_output_file}` (según tu configuración).

Este formato es útil para despliegue eficiente en entornos ligeros como servidores locales o integraciones con motores de inferencia optimizados.


In [ ]:
merged_model_path = "merged/"
gguf_output_abs_path = "/content/merged.gguf"

In [ ]:
# 运行 LoRA 合并脚本
#print("Starting LoRA merge...")
#!python merge_lora.py --base_model_name "{base_model_name}" --adapter_path "{adapter_path}" --output_path "{merged_model_path}"

# 转换为 GGUF
print("Starting GGUF conversion...")
!python convert_to_gguf.py --model_dir "/content/Qwen_Fine-tuning/merged" --output_file "{gguf_output_abs_path}" --out_type f16

In [ ]:
!pip install gradio

In [ ]:
!python /content/Qwen_Fine-tuning/app_gradio_qwen_ft.py

## (可选) 查看 train.py 的高级参数

运行下面的代码单元格可以显示 `train.py` 脚本支持的所有命令行参数及其说明和默认值。
如果您想覆盖默认设置（例如调整学习率、LoRA rank、保存步数等），可以在第 3 步运行训练时手动添加这些参数。


In [ ]:
!python train.py --help

# 6. (可选) 运行 GGUF 模型测试脚本


In [ ]:
# 检查 GGUF 文件是否存在
import os
if os.path.exists(gguf_output_abs_path):
    print(f"GGUF 文件存在: {gguf_output_abs_path}")
    print("运行 GGUF 模型测试...")

    print("安装 llama-cpp-python...")
    # 使用 CMAKE_ARGS 来启用 cuBLAS 以支持 GPU 加速
    !CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install -q llama-cpp-python
    !pip install -q llama-cpp-python

    # 构建测试命令
    test_gguf_command = (
        f"python test_gguf.py "
        f"--gguf_model_path \\\"{gguf_output_abs_path}\\\" "
        f"--n_gpu_layers -1 " # 尝试使用所有 GPU 层
        f"--chat_format qwen" # 明确指定 Qwen 格式
    )

    print("\\n--- Running GGUF Test Command ---")
    print(test_gguf_command)
    print("---------------------------------\\n")

    # 执行测试命令
    !{test_gguf_command}

else:
    print(f"错误：GGUF 文件未找到: {gguf_output_abs_path}")
    print("请确保前面的 GGUF 转换步骤已成功完成。")


In [ ]:
!pip install llama-cpp-python

In [ ]:
import os
import gradio as gr
from llama_cpp import Llama

# ⚙️ Ruta al modelo GGUF ya fusionado
GGUF_MODEL_PATH = "/content/merged.gguf"

# 🔧 Parámetros de carga (ajusta según RAM/VRAM disponible)
MODEL = Llama(
    model_path=GGUF_MODEL_PATH,
    n_ctx=2048,
    n_threads=os.cpu_count(),
    n_gpu_layers=33,  # usa 0 si no tienes GPU compatible, o ajusta a tu VRAM (ej: 33 para 8GB)
    verbose=False
)

# 💬 Generación de respuesta
def chat_with_model(prompt, temperature=0.7, max_tokens=256, top_p=0.95, repeat_penalty=1.15):
    try:
        # Si tu modelo usa template tipo ChatML (Qwen), agrega instrucción como:
        formatted_prompt = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"
        output = MODEL(
            formatted_prompt,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            repeat_penalty=repeat_penalty,
            stop=["<|im_end|>"]
        )
        return output["choices"][0]["text"].strip()
    except Exception as e:
        return f"❌ Error: {e}"

# 🧪 Interfaz Gradio
interface = gr.Interface(
    fn=chat_with_model,
    inputs=[
        gr.Textbox(lines=4, placeholder="Escribe algo...", label="Tu mensaje"),
        gr.Slider(minimum=0.1, maximum=1.5, value=0.7, label="Temperature"),
        gr.Slider(minimum=64, maximum=1024, value=256, label="Max Tokens"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.95, label="Top-p"),
        gr.Slider(minimum=1.0, maximum=2.0, value=1.15, label="Penalización de repetición"),
    ],
    outputs="text",
    title="🧠 Chat GGUF - Qwen Fine-tuned",
    description="Interactúa con tu modelo GGUF convertido desde QLoRA con Gradio",
    theme="default"
)

# 🟢 Ejecutar app
if __name__ == "__main__":
    interface.launch(share=True)
